In [1]:
# berikut adalah library untuk mengimplementasikan association rule mining
from mlxtend.preprocessing import TransactionEncoder # sebagai encoder untuk mengubah transaksi menjadi format array yang dapat digunakan untuk frequent itemsets
from mlxtend.frequent_patterns import apriori, association_rules # sebagai frequent itemsets dan menghasilkan rules dari frequent itemsets

# berikut adalah library tersebut untuk kegunaannya sebagai deteksi wajah
import cv2 # sebagai menangani pemrosesan gambar pengenalan wajah
import dlib # sebagai bagian dari face recognition.
import face_recognition # sebagai pengenalan wajah.
from mtcnn.mtcnn import MTCNN # sebagai deteksi wajah menggunakan.

# berikut adalah library tersebut untuk kegunaannya sebagai pembantu struktur data, yang digunakan juga untuk pre processing data
import numpy as np # sebagai mengelola numerik.
import pandas as pd # sebagai manipulasi data tabel.
import glob # sebagai pengolahan pencarian file.
from collections import defaultdict, Counter # sebagai mengelola koleksi data yang tidak teratur dan menghitung frekuensi elemen.
import os # sebagai operasi sistem file, seperti membaca, menulis, direktori.
from PIL import Image # sebagai membuka, dan menyimpan file gambar.
from numpy import savez_compressed # sebagai menyimpan array numpy dalam format kompresi.
from numpy import asarray # sebagai fungsi untuk mengkonversi data ke dalam array numpy.
from os import listdir # sebagai mendapatkan daftar file dalam direktori.
import shutil # sebagai operasi file dan direktori seperti copy dan move.
from sklearn.model_selection import train_test_split # sebagai library untuk membagi dataset menjadi set pelatihan dan pengujian.

# berikut adalah library untuk membantu merapihkan tampilan data pada output tampilan
from IPython.display import display_html # sebagai menampilkan gambar dalam notebook.
from IPython.display import display, HTML # sebagai menampilkan gambar notebook.
from matplotlib import pyplot as plt # sebagai membuat visualisasi grafik dan gambar.
from matplotlib.patches import Rectangle # sebagai menambahkan bentuk persegi panjang ke plot.
from matplotlib.patches import Circle # sebagai menambahkan bentuk lingkaran ke plot.

# berikut adalah library untuk bantuan dalam mengolah pembersihan pada dataset
import re # sebagai operasi pencarian string dengan suatu konteks tertentu contoh ingin mencari kata person dalam suatu value.
from itertools import combinations # sebagai menghasilkan semua kombinasi elemen dari iterable.
from ast import literal_eval # sebagai mengevaluasi string Python menjadi tipe data Python yang sesuai.

D:\myenv\Lib\site-packages\face_recognition_models\__init__.py:7: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  from pkg_resources import resource_filename
<frozen importlib._bootstrap>:488: DeprecationWarning: Type google._upb._message.MessageMapContainer uses PyType_Spec with a metaclass that has custom tp_new. This is deprecated and will no longer be allowed in Python 3.14.
<frozen importlib._bootstrap>:488: DeprecationWarning: Type google._upb._message.ScalarMapContainer uses PyType_Spec with a metaclass that has custom tp_new. This is deprecated and will no longer be allowed in Python 3.14.


## EXTRACT DATA USING FACE RECOGNITION

In [2]:
pengguna_user = '1'

In [3]:
# sebagai direktori penyimpanan untuk menyimpan embedding wajah yang dikenal.
embedding_dir = f'./known_embeddings{pengguna_user}'
os.makedirs(embedding_dir, exist_ok=True)

# sebagai fungsi untuk menyimpan embedding wajah yang terdeteksi dengan nama file yang sesuai dengan ID person.
def save_embedding(person_id, face_encoding):
    number = person_id.split('_')[-1]
    embedding_path = os.path.join(embedding_dir, f'person_{number}.npy')
    np.save(embedding_path, face_encoding)

# sebagai fungsi untuk load embedding wajah yang sudah tersimpan untuk digunakan dalam pencocokan wajah.
def load_embeddings():
    embeddings = {}
    # sebagai iterasi file embedding yang ada di direktori untuk load.
    for file in glob.glob(os.path.join(embedding_dir, '*.npy')):
        person_id = os.path.splitext(os.path.basename(file))[0]
        embeddings[person_id] = np.load(file)
    return embeddings

# sebagai fungsi untuk mencari kecocokan antara wajah yang sudah dikenal.
def find_best_match(face_encoding, known_embeddings):
    best_match_id = None
    min_distance = float('inf')

    # sebagai iterasi semua embedding yang dikenal untuk menghitung jarak (distance) antara encoding wajah yang dikomparasi dengan setiap embedding yang dikenal.
    for person_id, known_encoding in known_embeddings.items():
        distance = np.linalg.norm(face_encoding - known_encoding)
        if distance < min_distance:
            min_distance = distance
            number = person_id.split('_')[-1]
            best_match_id = number
    
    return best_match_id if min_distance < 0.6 else None  # Adjust threshold as needed

# sebagai fungsi untuk memproses setiap gambar, mendeteksi wajah, mengenali atau menyimpan ID baru, dan menyimpan wajah yang terdeteksi.
def process_image(image_path, output_base_dir, known_embeddings, photo_id):
    # sebagai membaca gambar dari file path.
    image = cv2.imread(image_path)
    face_locations = face_recognition.face_locations(image)
    face_encodings = face_recognition.face_encodings(image, face_locations)
    
    person_ids = []
    person_names = []

    # sebagai iterasi semua lokasi wajah dan encoding wajah yang terdeteksi di gambar.
    for i, (face_location, face_encoding) in enumerate(zip(face_locations, face_encodings)):
        top, right, bottom, left = face_location
        face_image = image[top:bottom, left:right]

        person_id = find_best_match(face_encoding, known_embeddings)

        # sebagai kondisi untuk menentukan apakah wajah yang terdeteksi merupakan wajah yang belum dikenal.
        if not person_id:  # If no match found, assign a new ID
            person_id = f'{len(known_embeddings) + 1}'
            save_embedding(person_id, face_encoding)
            known_embeddings[person_id] = face_encoding

        # sebagai membuat direktori baru untuk menyimpan wajah yang sesuai dengan ID yang dikenali atau baru dibuat.
        person_dir = os.path.join(output_base_dir, f'person_{person_id}')
        os.makedirs(person_dir, exist_ok=True)

        # sebagai menentukan nama file yang sesuai untuk menyimpan gambar wajah yang terdeteksi.
        face_index = len(glob.glob(os.path.join(person_dir, "*.jpg"))) + 1
        face_filename = os.path.join(person_dir, f'person_{person_id}_face_{face_index}.jpg')  # Updated filename
        cv2.imwrite(face_filename, face_image)

        # sebagai menyimpan ID wajah yang terdeteksi dengan format yang sesuai untuk identifikasi lebih lanjut.
        person_face_id = f"{person_id}_{face_index}"

        # sebagai menambahkan ID wajah yang terdeteksi ke daftar person_ids dan person_names untuk disimpan dalam dataset.
        person_ids.append(person_face_id)
        person_names.append(f'person_{person_id}_face_{face_index}')  # Updated naming

    # sebagai return value ke dictionary dengan ID foto, nama foto, ID orang, dan nama person yang terdeteksi.
    return {
        'photo_id': photo_id,
        'photo_name': os.path.basename(image_path),
        'person_id': ','.join(person_ids),
        'person': person_names
    }

# sebagai fungsi untuk memproses semua gambar dalam direktori yang diberikan, serta menggabungkan hasilnya dengan dataset yang ada.
def process_images(image_directory, output_base_dir, existing_dataset_path):
    image_paths = glob.glob(os.path.join(image_directory, '*.jpg'))
    
    # sebagai memeriksa apakah dataset yang ada sudah tersedia, dan jika ya, memuatnya serta melacak foto yang sudah diproses.
    if os.path.exists(existing_dataset_path):
        existing_df = pd.read_csv(existing_dataset_path)
        processed_photos = set(existing_df['photo_name'])
        start_photo_id = existing_df['photo_id'].max() + 1
    # sebagai inisialisasi dataset baru jika tidak ada dataset yang sudah ada.
    else:
        existing_df = pd.DataFrame()
        processed_photos = set()
        start_photo_id = 1
    
    dataset = []
    known_embeddings = load_embeddings()
    current_photo_id = start_photo_id

    # sebagai iterasi melalui semua gambar di direktori, memprosesnya jika belum pernah diproses, dan menambahkan hasilnya ke dataset.
    for image_path in image_paths:
        photo_name = os.path.basename(image_path)
        if photo_name in processed_photos:
            continue
        # sebagai memproses gambar yang belum diproses sebelumnya dan menambahkan hasilnya ke dalam dataset.
        result = process_image(image_path, output_base_dir, known_embeddings, current_photo_id)
        dataset.append(result)
        current_photo_id += 1

    # sebagai membuat DataFrame baru dari hasil pemrosesan dan menggabungkannya dengan dataset yang ada, kemudian menyimpannya ke file CSV.
    new_df = pd.DataFrame(dataset)
    combined_df = pd.concat([existing_df, new_df])
    if new_df.empty:
        combined_df.to_csv(existing_dataset_path, index=False)
        print("tidak ada tambahan foto baru")
    else:
        combined_df.to_csv(existing_dataset_path, index=False)
        print(f"ada tambahan foto baru sebanyak {len(new_df)} foto, data foto menjadi {len(combined_df)}")
    return combined_df

In [3]:
# sebagai variabel direktori gambar yang akan diproses, direktori output, dan path dataset yang ada.
image_dir = f'./pengguna {pengguna_user}'
output_base_dir = f'./clusterExtract/pengguna {pengguna_user}'
existing_dataset_path = f'./face_dataset{pengguna_user}_before_koreksi_sistem.csv'

In [ ]:
# sebagai memproses semua gambar di direktori yang diberikan dan menggabungkan hasilnya dengan dataset yang ada.
df_process = process_images(image_dir, output_base_dir, existing_dataset_path)

In [ ]:
# sebagai menghapus baris dengan nilai null dan baris di mana kolom 'person' adalah daftar kosong, kemudian menyimpan hasilnya ke dataset.
df_correct =  df_process.dropna()
df_correct = df_correct[df_correct['person'] != '[]']
df_correct

In [ ]:
# sebagai menyimpan dataset yang sudah dibersihkan ke dalam file CSV.
df_correct.to_csv(existing_dataset_path, index=False)

## CORRECTION TOOLS

In [4]:
df_correct = pd.read_csv(f'./face_dataset{pengguna_user}_before_koreksi_sistem.csv')
df_correct_temp = df_correct.copy()
df_correct_temp

,photo_id,photo_name,person_id,person
0,1,2013_08_30_19_35_IMG_3104.JPG,"2_1,1_1","['person_2_face_1', 'person_1_face_1']"
1,2,2013_08_30_19_37_IMG_2948.JPG,"1_2,2_2","['person_1_face_2', 'person_2_face_2']"
2,3,2013_08_30_19_37_IMG_2957.JPG,"1_3,2_3","['person_1_face_3', 'person_2_face_3']"
3,4,2014_01_01_09_20_IMG_2950.JPG,1_4,['person_1_face_4']
4,6,2014_01_01_10_18_IMG_2954.JPG,"2_4,1_5","['person_2_face_4', 'person_1_face_5']"
...,...,...,...,...
636,667,2021_01_02_17_22_IMG_4959.JPG,"1_548,26_52,21_24","['person_1_face_548', 'person_26_face_52', 'pe..."
637,668,2021_01_02_17_22_IMG_4960.JPG,"30_18,1_549,21_25","['person_30_face_18', 'person_1_face_549', 'pe..."
638,669,2021_01_02_17_22_IMG_4961.JPG,"21_26,26_53,1_550","['person_21_face_26', 'person_26_face_53', 'pe..."
639,670,2021_01_05_17_34_IMG_4995.JPG,21_27,['person_21_face_27']


In [6]:
# sebagai fungsi untuk mendapatkan indeks tertinggi dari gambar wajah dalam sebuah folder
def get_highest_face_index(directory):
    # Check if the directory exists
    if not os.path.exists(directory):
        return 0
    
    # Find files in the directory
    face_files = glob.glob(os.path.join(directory, "*.jpg"))
    
    # If no files are found, return 0
    if not face_files:
        return 0
    
    # Get the expected prefix from the directory name
    expected_prefix = os.path.basename(directory).split('_')[-1]
    
    # ekstrak angka pada nama file setelah kata face
    indices = []
    for face in face_files:
        # Extract the filename
        filename = os.path.basename(face)
        
        # Check if the filename starts with the expected prefix
        if filename.startswith(f"person_{expected_prefix}"):
            # menggunkan regex untuk mendapatkan angka setelah kata face
            match = re.search(r'face_(\d+)', filename)
            if match:
                indices.append(int(match.group(1)))
    
    # Return dengan indeks tertinggi
    return max(indices, default=0)


# sebagai fungsi untuk mengganti nama file gambar wajah agar konsisten dengan struktur folder
def rename_face_images(output_base_dir, old_name, new_name):
    # ekstrak id dari gambar lama dan untuk nama gambar barunya
    old_person_id = old_name.split('_')[1]
    new_person_id = new_name.split('_')[1]

    # mengambil gambar lama dengan mengakses path gambarnya pada folder
    old_path = os.path.join(output_base_dir, f'person_{new_person_id}', f"{old_name}.jpg")
    new_path = os.path.join(output_base_dir, f'person_{new_person_id}', f"{new_name}.jpg")

    # mengganti nama gambar
    os.rename(old_path, new_path)


# sebagai fungsi untuk memeriksa konsistensi nama file gambar dengan folder yang benar, dan memperbarui df jika ditemukan inkonsistensi
def check_folder_consistency(output_base_dir, df):
    print("Checking folder consistency...")
    count = 0

    # looping untuk setiap cluster folder wajah individu
    for person_folder in glob.glob(os.path.join(output_base_dir, 'person_*')):
        dir_folder = person_folder
        # ekstrak id person yang ada pada nama difolder tersebut
        person_id = os.path.basename(person_folder).split('_')[-1]
        print(f"Checking folder: {person_folder}")

        # looping untuk setiap gambar pada folder wajah individu
        for face_image in glob.glob(os.path.join(person_folder, '*.jpg')):
            #ekstra path name gambar sehingga mendaptkan nama folder cluster person yang sesuai serta mendapatkan nama asli dari gambar
            face_name = os.path.basename(face_image)
            # mengambil nama dasar dari gambar untuk memeriksa konsistensi
            base_name = face_name.split('_')[0]
            # mengambil id person yang ada pada nama gambar
            person_id_name = face_name.split('_')[1]
            # mengambil nama file gambar untuk diperiksa kesesuaiannya
            face_name = f"{base_name}_{person_id_name}"
            # mengambil nama id person yang diharapkan sesuai dengan nama folder
            expected_person_id = f'person_{person_id}'

            # jika nma asli dari gambar tidak sama dengan nama folder cluster seperti person_24 != person_3
            if face_name != expected_person_id:
                # mengambil id person lama yang ada pada nama gambar
                old_person_id = face_name.split('_')[1]
                # mengambil bagian kedua dari id person pada nama gambar
                old_person_id_2 = face_image.split('_')[4][:-4]
                # mengambil id gambar lama yang ditemukan tidak sesuai dengan folder
                old_img_id = f'{old_person_id}_{old_person_id_2}'
                print(f"File person_{old_person_id} does not match expected person_id {expected_person_id}")
                # mengambil id gambar baru yang dihasilkan setelah memeriksa indeks wajah tertinggi di folder
                expected_person_id_2 = get_highest_face_index(dir_folder) + 1
                # mengambil id gambar baru yang akan digunakan untuk menggantikan id gambar lama
                new_img_id = f'{person_id}_{expected_person_id_2}'

                # sebagai fungsi untuk mencocokkan id person lama dengan baris di DataFrame
                def match_person_id(row, old_img_id):
                    return old_img_id in row['person_id'].split(',')

                # sebagai dataframe yang berisi baris yang sesuai dengan id person lama
                matching_rows = df[df.apply(lambda row: match_person_id(row, old_img_id), axis=1)]
                # sebagai kondisi jika baris yang sesuai ditemukan di DataFrame
                if not matching_rows.empty:
                    old_person_name = f'{face_name}_face_{old_person_id_2}'
                    new_person_name = f'{expected_person_id}_face_{expected_person_id_2}'
                    # sebagai langkah untuk mengganti nama file gambar di folder
                    rename_face_images(output_base_dir, old_person_name, new_person_name)

                    # sebagai langkah untuk memperbarui DataFrame dengan nama dan id person baru
                    for index, row in matching_rows.iterrows():
                        person_list = eval(row['person'])  # Convert the string representation of list back to list
                        # sebagai daftar id person yang ada pada baris DataFrame
                        person_id_list = row['person_id'].split(',')
                        # sebagai daftar person yang diperbarui di DataFrame
                        updated_person_list = [new_person_name if p == old_person_name else p for p in person_list]
                        # sebagai daftar id person yang diperbarui di DataFrame
                        updated_person_id_list = [new_img_id if p == old_img_id else p for p in person_id_list]
                        # sebagai id person yang diperbarui dalam bentuk string yang akan disimpan di DataFrame
                        update_df_id = ','.join(updated_person_id_list)
                        df.at[index, 'person'] = str(updated_person_list)
                        df.at[index, 'person_id'] = str(update_df_id)
                        print(f"Updated entry: {old_person_name} -> {new_person_name}")
                        print(f"From photo_id: {matching_rows['photo_id'].to_string(index=False)}")
    
    return df

In [7]:
# sebagai menjalankan fungsi cek konsistensi folder dan df
df_correct_after = check_folder_consistency(output_base_dir, df_correct_temp)

Checking folder consistency...
Checking folder: ./clusterExtract/pengguna 1\person_1
File person_10 does not match expected person_id person_1
Updated entry: person_10_face_32 -> person_1_face_552
From photo_id: 37
File person_10 does not match expected person_id person_1
Updated entry: person_10_face_35 -> person_1_face_553
From photo_id: 39
File person_10 does not match expected person_id person_1
Updated entry: person_10_face_36 -> person_1_face_554
From photo_id: 43
File person_10 does not match expected person_id person_1
Updated entry: person_10_face_37 -> person_1_face_555
From photo_id: 44
File person_10 does not match expected person_id person_1
Updated entry: person_10_face_38 -> person_1_face_556
From photo_id: 47
File person_10 does not match expected person_id person_1
Updated entry: person_10_face_39 -> person_1_face_557
From photo_id: 48
File person_10 does not match expected person_id person_1
Updated entry: person_10_face_52 -> person_1_face_558
From photo_id: 158
File

In [8]:
# sebagai konvert df menjadi csv yang disimpan pada folder
df_correct_after.to_csv(f'./face_dataset{pengguna_user}_after_koreksi_sistem.csv', index=False)

In [9]:
import ast

# Fungsi untuk menghitung kemunculan pola 'person_2_face_' di setiap elemen list
def count_person_2_face(person_list_str):
    # Konversi string yang menyerupai list menjadi list Python sesungguhnya
    person_list = ast.literal_eval(person_list_str)
    return sum(bool(re.match(r'person_2_face_\d+', person)) for person in person_list)

# Terapkan fungsi ke kolom 'person' dan jumlahkan hasilnya
total_count_before = df_correct['person'].apply(count_person_2_face).sum()
total_count_after = df_correct_after['person'].apply(count_person_2_face).sum()

print(f'JUMLAH Before: {total_count_before}')
print(f'JUMLAH After: {total_count_after}')

JUMLAH Before: 191
JUMLAH After: 271


### LOAD DATA 

In [10]:
df = pd.read_csv(f'./face_dataset{pengguna_user}_after_koreksi_sistem.csv')
df_temp = df_correct_after.copy()
df_temp

,photo_id,photo_name,person_id,person
0,1,2013_08_30_19_35_IMG_3104.JPG,"2_1,1_1","['person_2_face_1', 'person_1_face_1']"
1,2,2013_08_30_19_37_IMG_2948.JPG,"1_2,2_2","['person_1_face_2', 'person_2_face_2']"
2,3,2013_08_30_19_37_IMG_2957.JPG,"1_3,2_3","['person_1_face_3', 'person_2_face_3']"
3,4,2014_01_01_09_20_IMG_2950.JPG,1_4,['person_1_face_4']
4,6,2014_01_01_10_18_IMG_2954.JPG,"2_4,1_5","['person_2_face_4', 'person_1_face_5']"
...,...,...,...,...
636,667,2021_01_02_17_22_IMG_4959.JPG,"1_548,7_52,21_24","['person_1_face_548', 'person_7_face_52', 'per..."
637,668,2021_01_02_17_22_IMG_4960.JPG,"7_57,1_549,21_25","['person_7_face_57', 'person_1_face_549', 'per..."
638,669,2021_01_02_17_22_IMG_4961.JPG,"21_26,7_53,1_550","['person_21_face_26', 'person_7_face_53', 'per..."
639,670,2021_01_05_17_34_IMG_4995.JPG,26_65,['person_26_face_65']


## DATA CLEANING

In [11]:
# sebagai memodifikasi elemen dalam kolom person agar hanya menyisakan 2 bagian pertama dari nama setelah underscore kedua
def remove_after_second_underscore(person_list):
    modified_list = []
    for person in person_list:
        parts = person.split('_')
        if len(parts) > 2:
            modified_list.append('_'.join(parts[:2]))  # Hanya menyimpan dua bagian pertama
        else:
            modified_list.append(person)
    return modified_list

# sebagai memodifikasi elemen dalam kolom person agar hanya menyisakan bagian 1 dari nama setelah underscore pertama
def remove_after_first_underscore(person_list):
    modified_list = []
    for person in person_list:
        parts = person.split('_')
        if len(parts) > 1:
            modified_list.append(parts[0])  # Hanya menyimpan bagian pertama
        else:
            modified_list.append(person)
    return modified_list

# sebagai memodifikasi kolom 'person' dalam df dengan mengaplikasikan perubahan pada elemen-elemen sesuai dengan aturan yang ditentukan
def modify_person_column(current_df):
    # Fungsi untuk memproses setiap elemen dalam kolom 'person' dan memodifikasinya sesuai dengan aturan yang kedua
    def process_person_column_2(person_str):
        person_list = person_str.split(',')
        modified_person_list = remove_after_second_underscore(person_list)
        return str([f"{p.strip()}" for p in modified_person_list])

    # Fungsi untuk memproses setiap elemen dalam kolom 'person_id' dan memodifikasinya sesuai dengan aturan yang pertama
    def process_person_column_1(person_str):
        person_list = person_str.split(',')
        modified_person_list = remove_after_first_underscore(person_list)
        return ', '.join(modified_person_list)
    
    # Mengaplikasikan modifikasi pada kolom 'person' dan 'person_id' dalam DataFrame
    current_df['person'] = current_df['person'].apply(lambda x: process_person_column_2(x.strip("[]").replace("'", "")))
    current_df['person_id'] = current_df['person_id'].apply(process_person_column_1)
    return current_df

# sebagai menghapus nilai tertentu dari daftar
def remove_values_from_list(lst, values):
    return [item for item in lst if item not in values]

# sebagai menghapus nilai tertentu dari string yang dipisahkan oleh koma
def remove_values_from_string(s, values):
    return ', '.join(item for item in s.split(', ') if item not in values)

# sebagai mengkonversi daftar ke dalam bentuk string yang sesuai
def list_to_string(lst):
    return str(lst).replace('"', '')

# sebagai membersihkan data dalam df dengan menghapus nilai-nilai tertentu dari kolom 'person' dan 'person_id'
def clean_data(df, values_to_remove, ids_to_remove):
    df['person'] = df['person'].apply(lambda x: remove_values_from_list(x, values_to_remove))
    df['person_id'] = df['person_id'].apply(lambda x: remove_values_from_string(x, ids_to_remove))
    df['person'] = df['person'].apply(list_to_string)
    return df

# Nilai-nilai yang akan dihapus berdasarkan user yang mengakses
if pengguna_user == '1':
    values_to_remove = {'person_40'}
    ids_to_remove = {'40'}
else:
    values_to_remove = {'person_999'}
    ids_to_remove = {'999'}


In [12]:
# sebagai memodifikasi kolom person dan person_id dalam df_temp
df_cleaned = modify_person_column(df_temp)

# sebagai mengkonversi string list dalam kolom person menjadi list yang lebih layak
df_cleaned['person'] = df_cleaned['person'].apply(literal_eval)

# sebagai membersihkan data dalam df_cleaned dengan menghapus nilai-nilai dan ID yang tidak diinginkan berdasarkan nilai yang telah ditentukan
df_cleaned = clean_data(df_cleaned, values_to_remove, ids_to_remove)


In [13]:
df_cleaned.to_csv(f'./face_dataset{pengguna_user}_after_koreksi_sistem_clean.csv', index=False)
df_cleaned

,photo_id,photo_name,person_id,person
0,1,2013_08_30_19_35_IMG_3104.JPG,"2, 1","['person_2', 'person_1']"
1,2,2013_08_30_19_37_IMG_2948.JPG,"1, 2","['person_1', 'person_2']"
2,3,2013_08_30_19_37_IMG_2957.JPG,"1, 2","['person_1', 'person_2']"
3,4,2014_01_01_09_20_IMG_2950.JPG,1,['person_1']
4,6,2014_01_01_10_18_IMG_2954.JPG,"2, 1","['person_2', 'person_1']"
...,...,...,...,...
636,667,2021_01_02_17_22_IMG_4959.JPG,"1, 7, 21","['person_1', 'person_7', 'person_21']"
637,668,2021_01_02_17_22_IMG_4960.JPG,"7, 1, 21","['person_7', 'person_1', 'person_21']"
638,669,2021_01_02_17_22_IMG_4961.JPG,"21, 7, 1","['person_21', 'person_7', 'person_1']"
639,670,2021_01_05_17_34_IMG_4995.JPG,26,['person_26']


In [14]:
import ast

# sebagai mengecek apakah pembersihan data berhasil atau tidak
def count_person_2_face(person_list_str):
    # Konversi string yang menyerupai list menjadi list Python sesungguhnya
    person_list = ast.literal_eval(person_list_str)
    return sum(bool(re.match(r'person_40_face_\d+', person)) for person in person_list)

# Terapkan fungsi ke kolom 'person' dan jumlahkan hasilnya
total_count_before = df_correct_after['person'].apply(count_person_2_face).sum()
total_count_after = df_cleaned['person'].apply(count_person_2_face).sum()

print(f'JUMLAH Before: {total_count_before}')
print(f'JUMLAH After: {total_count_after}')

JUMLAH Before: 36
JUMLAH After: 0


#### ASSOCIATION RULES

In [15]:
df_cleaned = pd.read_csv(f'./face_dataset{pengguna_user}_after_koreksi_sistem_clean.csv')
df_cleaned

,photo_id,photo_name,person_id,person
0,1,2013_08_30_19_35_IMG_3104.JPG,"2, 1","['person_2', 'person_1']"
1,2,2013_08_30_19_37_IMG_2948.JPG,"1, 2","['person_1', 'person_2']"
2,3,2013_08_30_19_37_IMG_2957.JPG,"1, 2","['person_1', 'person_2']"
3,4,2014_01_01_09_20_IMG_2950.JPG,1,['person_1']
4,6,2014_01_01_10_18_IMG_2954.JPG,"2, 1","['person_2', 'person_1']"
...,...,...,...,...
636,667,2021_01_02_17_22_IMG_4959.JPG,"1, 7, 21","['person_1', 'person_7', 'person_21']"
637,668,2021_01_02_17_22_IMG_4960.JPG,"7, 1, 21","['person_7', 'person_1', 'person_21']"
638,669,2021_01_02_17_22_IMG_4961.JPG,"21, 7, 1","['person_21', 'person_7', 'person_1']"
639,670,2021_01_05_17_34_IMG_4995.JPG,26,['person_26']


In [16]:
# Function to safely evaluate a string to a list
# menghindari kesalahan jika nilai bukanlah string atau tidak
def safe_eval(val):
    if isinstance(val, str):
        return eval(val)
    return val

In [17]:
def find_common_rules(df_list, min_support=0.03, min_threshold=0.9):
    # Membuat dictionary untuk menyimpan aturan dari setiap DataFrame
    rules_dict = {}
    
    # Daftar nama data
    # data_names = ['data_1_4', 'data_3_4', 'data_1_2', 'data_full']
    data_names = ['data_full']
    
    # Jika terdapat lebih dari satu nama data
    if len(data_names) > 1:
        # Iterasi melalui setiap DataFrame dalam df_list
        for i, df in enumerate(df_list):
            # Membuat TransactionEncoder untuk mempersiapkan data transaksi
            te = TransactionEncoder()
            # Mengubah kolom 'person' menjadi list of lists menggunakan safe_eval
            transactions = df['person'].apply(safe_eval).tolist()
            # Transformasi transaksi ke dalam bentuk array
            te_ary = te.fit(transactions).transform(transactions)
            # Membuat DataFrame dari array yang sudah ditransformasikan
            df_transformed = pd.DataFrame(te_ary, columns=te.columns_)
            # Menemukan itemset yang sering muncul menggunakan apriori
            frequent_itemsets = apriori(df_transformed, min_support=min_support, use_colnames=True)
            # Menemukan aturan asosiasi dari itemset yang sering muncul
            rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=min_threshold)
            # Menyimpan nama sumber data asli dalam kolom 'source'
            rules['source'] = data_names[i]
            # Menyimpan aturan ke dalam dictionary dengan kunci indeks
            rules_dict[i] = rules

        # Membandingkan rules antar pasangan df untuk menemukan rules yang sama
        for i in range(len(rules_dict) - 1):
            for j in range(i + 1, len(rules_dict)):
                # Menggabungkan dua DataFrame aturan berdasarkan 'antecedents' dan 'consequents'
                common_rules = pd.merge(rules_dict[i], rules_dict[j], on=['antecedents', 'consequents'], suffixes=('_x', '_y'))
                if not common_rules.empty:
                    # Membuat DataFrame baru dengan kolom yang diinginkan
                    result = pd.DataFrame({
                        'antecedents': common_rules['antecedents'],
                        'consequents': common_rules['consequents'],
                        'antecedent support': common_rules['antecedent support_x'],
                        'consequent support': common_rules['consequent support_x'],
                        'support': common_rules['support_x'],
                        'confidence': common_rules['confidence_x'],
                        'lift': common_rules['lift_x'],
                    })
                    # Menggabungkan nama sumber data asli
                    result['sources'] = data_names[i] + ', ' + data_names[j]
                    # Mengambil photo_id dari DataFrame kedua dan menyimpannya dalam kolom 'photo_id'
                    photo_id_j = df_list[j]['photo_id'].max()
                    result['photo_id'] = str(photo_id_j)
                    return result
    else:
        # Jika hanya ada satu nama data
        te = TransactionEncoder()
        # Mengubah kolom 'person' menjadi list of lists menggunakan safe_eval dari df_list pertama
        transactions = df_list[0]['person'].apply(safe_eval).tolist()
        # Transformasi transaksi ke dalam bentuk array
        te_ary = te.fit(transactions).transform(transactions)
        # Membuat DataFrame dari array yang sudah ditransformasikan
        df_transformed = pd.DataFrame(te_ary, columns=te.columns_)
        # Menemukan itemset yang sering muncul menggunakan apriori
        frequent_itemsets = apriori(df_transformed, min_support=min_support, use_colnames=True)
        # Menemukan aturan asosiasi dari itemset yang sering muncul
        rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=min_threshold)
        
        # Membuat DataFrame hasil dengan kolom yang diinginkan
        result = pd.DataFrame({
            'antecedents': rules['antecedents'],
            'consequents': rules['consequents'],
            'antecedent support': rules['antecedent support'],
            'consequent support': rules['consequent support'],
            'support': rules['support'],
            'confidence': rules['confidence'],
            'lift': rules['lift'],
        })
        # Menyimpan nama sumber data asli
        result['sources'] = data_names[0]
        # Mengambil photo_id dari DataFrame pertama dan menyimpannya dalam kolom 'photo_id'
        result['photo_id'] = df_list[0]['photo_id'].max()
        return result
            
    # Mengembalikan None jika tidak ada aturan yang sama ditemukan
    return None


In [18]:
# memuat rules yang sudah ada dari file CSV
# jika file ada, baca aturan dari file dan return df
# jika file tidak ada, return df kosong
def load_existing_rules(rules_path):
    if os.path.exists(rules_path):
        rules_df = pd.read_csv(rules_path)
        return rules_df
    else:
        return pd.DataFrame()  # Return an empty DataFrame if no rules exist

# menyimpan rules baru ke file CSV
# jika rules_df adalah None, buat df kosong dengan kolom yang ditentukan
# jika rules_df tidak kosong, simpan df ke file
def save_new_rules(rules_df, rules_path):
    # Define the columns for an empty DataFrame
    columns = [
        'antecedents', 'consequents', 'antecedent support',
        'consequent support', 'support', 'confidence', 'lift',
        'sources', 'photo_id'
    ]
    
    if rules_df is None:
        rules_df = pd.DataFrame(columns=columns)
    
    # Check if rules_df is not empty
    if not rules_df.empty:
        rules_save = rules_df
    else:
        rules_save = pd.DataFrame(columns=columns)
    
    # Save the DataFrame to a CSV file
    rules_save.to_csv(rules_path, index=False)

# sebagai memeriksa perubahan pada aturan antara rule baru dan rules yang sudah ada
# jika aturan baru ada dan sama dengan aturan lama, return True
# jika tidak, aturan baru ditemukan dan disimpan, dan return True
def check_rules_changes(rules_fix, existing_rules_df):
    if rules_fix is not None:
        if (rules_fix[['antecedents', 'consequents']].equals(existing_rules_df[['antecedents', 'consequents']])):
            print("Ada foto tambahan, tetapi rules baru sama dengan rules lama.")
            return True
        else:
            print(f"Rules baru ditemukan dan disimpan. Menggunakan dataset dari {rules_fix['sources'].iloc[0]}.")
            return True
    else:
        print("Ada foto tambahan, tetapi rules baru sama dengan rules lama.")
        return False

# sebagai memeriksa dan memperbarui rules berdasarkan dataset
# jika peningkatan jumlah foto melebihi ambang batas
# simpan rules baru jika ada perubahan signifikan, jika tidak, return aturan lama
def check_and_update_rules(df, rules_path, min_support, min_threshold):
    # Load current dataset
    modified_df_clean = df

    # Load existing rules
    existing_rules_df = load_existing_rules(rules_path)

    # Determine number of photos in the dataset
    current_photo_count = modified_df_clean['photo_id'].nunique()

    # Determine number of photos when the last rules were created
    if existing_rules_df.empty:
        previous_photo_count = 0
    else:
        previous_photo_count = existing_rules_df['photo_id'].max()

    # Calculate the increase in photo count
    photo_count_increase = current_photo_count - previous_photo_count

    # Calculate the percentage increase
    percentage_increase = (photo_count_increase / previous_photo_count) * 100 if previous_photo_count > 0 else 100

    # jika peningkatan jumlah foto melebihi 30%, bagi data
    # simpan aturan baru jika ada perubahan signifikan atau gunakan aturan lama jika tidak ada perubahan signifikan
    if percentage_increase > 0.3:
        data_1_4, _ = train_test_split(modified_df_clean, train_size=0.25, random_state=42)
        data_3_4, _ = train_test_split(modified_df_clean, train_size=0.75, random_state=42)
        data_1_2, _ = train_test_split(modified_df_clean, train_size=0.50, random_state=42)
        data_full = modified_df_clean
        # df_list = [data_1_4, data_3_4, data_1_2, data_full]
        df_list = [data_full]
        rules_fix = find_common_rules(df_list, min_support=min_support, min_threshold=min_threshold)

        # simpan aturan baru sementara
        rules_fix.to_csv('./rules_temp.csv', index=False)
        if existing_rules_df.empty:
            print("Rules baru telah dibuat")
            save_new_rules(rules_fix, rules_path)
            return rules_fix, modified_df_clean
        else:
            rules_fix = pd.read_csv('./rules_temp.csv')
            if check_rules_changes(rules_fix, existing_rules_df):
                # Save the new rules
                save_new_rules(rules_fix, rules_path)
                return rules_fix, modified_df_clean
            else:
                existing_rules_df["photo_id"] = rules_fix["photo_id"].iloc[0]
                return existing_rules_df, modified_df_clean
    else:
        # Otherwise, load and return the existing rules
        print("Menggunakan rules lama. Tidak ada perubahan signifikan dalam jumlah foto.")
        return existing_rules_df, modified_df_clean


In [19]:
# Example usage
support_rules = 0.03
confidence_rules = 0.7
path_rules = f'./rules_file/pengguna {pengguna_user}/filtered_rules{pengguna_user}_{support_rules}_{confidence_rules}_ORI_test.csv'
rules_df, data_photo = check_and_update_rules(df_cleaned, rules_path=path_rules, min_support=support_rules, min_threshold=confidence_rules)
filtered_rules = rules_df
print(f"Rules yang dipilih berasal dari {rules_df["sources"].iloc[0]}  membuat sebanyak {len(filtered_rules)} rules")
filtered_rules

Menggunakan rules lama. Tidak ada perubahan signifikan dalam jumlah foto.
Rules yang dipilih berasal dari data_full  membuat sebanyak 97 rules


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,sources,photo_id
0,frozenset({'person_10'}),frozenset({'person_1'}),0.063963,0.804992,0.060842,0.951220,1.181651,data_full,671
1,frozenset({'person_13'}),frozenset({'person_1'}),0.043682,0.804992,0.043682,1.000000,1.242248,data_full,671
2,frozenset({'person_14'}),frozenset({'person_1'}),0.042122,0.804992,0.039002,0.925926,1.150230,data_full,671
3,frozenset({'person_19'}),frozenset({'person_1'}),0.040562,0.804992,0.032761,0.807692,1.003354,data_full,671
4,frozenset({'person_2'}),frozenset({'person_1'}),0.411856,0.804992,0.296412,0.719697,0.894042,data_full,671
...,...,...,...,...,...,...,...,...,...
92,"frozenset({'person_1', 'person_4', 'person_9'})",frozenset({'person_5'}),0.059282,0.113885,0.059282,1.000000,8.780822,data_full,671
93,"frozenset({'person_5', 'person_4'})","frozenset({'person_1', 'person_9'})",0.068643,0.110764,0.059282,0.863636,7.797055,data_full,671
94,"frozenset({'person_1', 'person_4'})","frozenset({'person_5', 'person_9'})",0.065523,0.095164,0.059282,0.904762,9.507416,data_full,671
95,"frozenset({'person_4', 'person_9'})","frozenset({'person_5', 'person_1'})",0.063963,0.096724,0.059282,0.926829,9.582219,data_full,671


### KLASIFIKASI KORELASI WAJAH INDIVIDU ANTAR FOTO

#### LOAD RULES IF NEEDS

In [20]:
# sebagai mengambil file berdasarkan dari parameter rules nya
pengguna_user = '1'
support_rules = 0.03
confidence_rules = 0.7
path_rules = f'./rules_file/pengguna {pengguna_user}/filtered_rules{pengguna_user}_{support_rules}_{confidence_rules}_ORI_test.csv'
filtered_rules = pd.read_csv(path_rules)


In [21]:
# sebagai mengambil file csv dataset dari hasil extract data yang berisi informasi wajah yang sudah bersih
data_photo = pd.read_csv(f'./face_dataset{pengguna_user}_after_koreksi_sistem_clean.csv')
data_photo

,photo_id,photo_name,person_id,person
0,1,2013_08_30_19_35_IMG_3104.JPG,"2, 1","['person_2', 'person_1']"
1,2,2013_08_30_19_37_IMG_2948.JPG,"1, 2","['person_1', 'person_2']"
2,3,2013_08_30_19_37_IMG_2957.JPG,"1, 2","['person_1', 'person_2']"
3,4,2014_01_01_09_20_IMG_2950.JPG,1,['person_1']
4,6,2014_01_01_10_18_IMG_2954.JPG,"2, 1","['person_2', 'person_1']"
...,...,...,...,...
636,667,2021_01_02_17_22_IMG_4959.JPG,"1, 7, 21","['person_1', 'person_7', 'person_21']"
637,668,2021_01_02_17_22_IMG_4960.JPG,"7, 1, 21","['person_7', 'person_1', 'person_21']"
638,669,2021_01_02_17_22_IMG_4961.JPG,"21, 7, 1","['person_21', 'person_7', 'person_1']"
639,670,2021_01_05_17_34_IMG_4995.JPG,26,['person_26']


In [22]:
df_photos = data_photo

In [23]:
filtered_rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,sources,photo_id
0,frozenset({'person_10'}),frozenset({'person_1'}),0.063963,0.804992,0.060842,0.951220,1.181651,data_full,671
1,frozenset({'person_13'}),frozenset({'person_1'}),0.043682,0.804992,0.043682,1.000000,1.242248,data_full,671
2,frozenset({'person_14'}),frozenset({'person_1'}),0.042122,0.804992,0.039002,0.925926,1.150230,data_full,671
3,frozenset({'person_19'}),frozenset({'person_1'}),0.040562,0.804992,0.032761,0.807692,1.003354,data_full,671
4,frozenset({'person_2'}),frozenset({'person_1'}),0.411856,0.804992,0.296412,0.719697,0.894042,data_full,671
...,...,...,...,...,...,...,...,...,...
92,"frozenset({'person_1', 'person_4', 'person_9'})",frozenset({'person_5'}),0.059282,0.113885,0.059282,1.000000,8.780822,data_full,671
93,"frozenset({'person_5', 'person_4'})","frozenset({'person_1', 'person_9'})",0.068643,0.110764,0.059282,0.863636,7.797055,data_full,671
94,"frozenset({'person_1', 'person_4'})","frozenset({'person_5', 'person_9'})",0.065523,0.095164,0.059282,0.904762,9.507416,data_full,671
95,"frozenset({'person_4', 'person_9'})","frozenset({'person_5', 'person_1'})",0.063963,0.096724,0.059282,0.926829,9.582219,data_full,671


In [24]:
# Apply safe_eval to the person column in df_photos
# memastikan bahwa data dalam kolom person yang sebelumnya disimpan sebagai string bisa diproses sebagai list yang valid
df_photos['person'] = df_photos['person'].apply(safe_eval)

# Apply safe_eval to the 'antecedents' and 'consequents' columns in filtered_rules
# memastikan bahwa data dalam kolom ini dapat diproses dengan benar
filtered_rules['antecedents'] = filtered_rules['antecedents'].apply(safe_eval)
filtered_rules['consequents'] = filtered_rules['consequents'].apply(safe_eval)


In [25]:
filtered_rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,sources,photo_id
0,(person_10),(person_1),0.063963,0.804992,0.060842,0.951220,1.181651,data_full,671
1,(person_13),(person_1),0.043682,0.804992,0.043682,1.000000,1.242248,data_full,671
2,(person_14),(person_1),0.042122,0.804992,0.039002,0.925926,1.150230,data_full,671
3,(person_19),(person_1),0.040562,0.804992,0.032761,0.807692,1.003354,data_full,671
4,(person_2),(person_1),0.411856,0.804992,0.296412,0.719697,0.894042,data_full,671
...,...,...,...,...,...,...,...,...,...
92,"(person_1, person_4, person_9)",(person_5),0.059282,0.113885,0.059282,1.000000,8.780822,data_full,671
93,"(person_5, person_4)","(person_1, person_9)",0.068643,0.110764,0.059282,0.863636,7.797055,data_full,671
94,"(person_1, person_4)","(person_5, person_9)",0.065523,0.095164,0.059282,0.904762,9.507416,data_full,671
95,"(person_4, person_9)","(person_1, person_5)",0.063963,0.096724,0.059282,0.926829,9.582219,data_full,671


In [26]:
def normalize_rule(antecedents, consequents):
    """ Normalize the rule to a canonical form. """
    # Untuk memastikan bahwa rules memiliki representasi yang konsisten, meskipun urutan antecedents atau consequents berbeda.
    return (frozenset(antecedents), frozenset(consequents))

def group_correlated_rules(rules_df):
    grouped_rules = []
    used_indices = set()
    rule_map = {}

    # Step 1: Group correlated rules
    # Untuk mengelompokkan rules yang berkorelasi, yaitu rules yang berbagi antecedents dan consequents yang saling beririsan.
    for i, row_i in rules_df.iterrows():
        if i in used_indices:
            continue
        
        new_antecedents = set(row_i['antecedents'])
        new_consequents = set(row_i['consequents'])

        # Sebagai loop utama untuk membandingkan setiap rules dengan rules lainnya.
        # Untuk menemukan rules yang memiliki antecedents atau consequents yang beririsan dengan rules yang sedang diproses.
        # Analoginya seperti menggabungkan kelompok-kelompok orang yang memiliki anggota yang sama.
        # rules diperiksa dengan rules lain yang belum digunakan untuk melihat apakah mereka memiliki hubungan (beririsan) dengan antecedents dan consequents yang sama.
        # Mulai dari rules pertama, bandingkan dengan rules lainnya, lalu gabungkan antecedents dan consequents yang beririsan.
        for j, row_j in rules_df.iterrows():
            # Sebagai kondisi untuk melewatkan perbandingan dengan aturan yang sama atau yang sudah digunakan.
            if i == j or j in used_indices:
                continue

            # Sebagai langkah untuk memperbarui antecedents dan consequents baru jika ditemukan rules yang berkorelasi.
            # Untuk memperluas kelompok antecedents dan consequents dengan menggabungkan rules yang beririsan.
            # seperti memperbesar kelompok orang yang terkait dengan menambahkan anggota baru.
            # gabungkan antecedents dan consequents dari rules yang beririsan.
            # memeriksa setiap rules, gabungkan jika ada hubungan, dan tandai rules tersebut.
            if not new_antecedents.isdisjoint(row_j['antecedents']) and not new_consequents.isdisjoint(row_j['consequents']):
                new_antecedents.update(row_j['antecedents'])
                new_consequents.update(row_j['consequents'])
                used_indices.add(j)
        
        # Normalize the rule
        rule = normalize_rule(new_antecedents, new_consequents)
        grouped_rules.append(rule)
        used_indices.add(i)

    # Step 2: Remove inverse rules
    # Untuk menghindari duplikasi aturan dalam bentuk terbalik.
    final_rules = []
    seen_rules = set()

    # Sebagai menghapus rules yang merupakan kebalikan dari rules yang sudah ada.
    # Untuk menjaga agar rules yang disimpan unik dan tidak redundan.
    # memeriksa apakah kebalikan dari rules sudah ada, jika belum, tambahkan rules tersebut ke dalam daftar.
    # iterasi melalui rules yang dikelompokkan, periksa kebalikannya, dan tambahkan ke daftar final jika belum ada.
    for rule in grouped_rules:
        antecedents, consequents = rule
        inverse_rule = normalize_rule(consequents, antecedents)
        
        # Add rule only if its inverse hasn't been seen
        # Sebagai mekanisme untuk memastikan aturan yang ditambahkan tidak memiliki pasangan kebalikannya yang sudah ada.
        if inverse_rule not in seen_rules:
            final_rules.append(rule)
            seen_rules.add(rule)
    
    return final_rules


In [27]:
# Group correlated rules
grouped_rules = group_correlated_rules(filtered_rules)
grouped_rules

[(frozenset({'person_1', 'person_10', 'person_34', 'person_35', 'person_8'}),
  frozenset({'person_1', 'person_10', 'person_34', 'person_35', 'person_8'})),
 (frozenset({'person_13'}), frozenset({'person_1'})),
 (frozenset({'person_14', 'person_19'}),
  frozenset({'person_1', 'person_14', 'person_19'})),
 (frozenset({'person_19'}), frozenset({'person_1'})),
 (frozenset({'person_2'}), frozenset({'person_1'})),
 (frozenset({'person_26'}), frozenset({'person_1'})),
 (frozenset({'person_3'}), frozenset({'person_1'})),
 (frozenset({'person_34', 'person_35'}),
  frozenset({'person_1', 'person_34', 'person_35'})),
 (frozenset({'person_35'}), frozenset({'person_1'})),
 (frozenset({'person_1', 'person_4', 'person_5', 'person_9'}),
  frozenset({'person_1', 'person_5', 'person_9'})),
 (frozenset({'person_5'}), frozenset({'person_1'})),
 (frozenset({'person_7'}), frozenset({'person_1'})),
 (frozenset({'person_8'}), frozenset({'person_1'})),
 (frozenset({'person_9'}), frozenset({'person_1'})),
 (fr

In [28]:
grup_rule = pd.DataFrame(grouped_rules, columns=['IF', 'THEN'])
grup_rule

,IF,THEN
0,"(person_34, person_1, person_10, person_35, pe...","(person_34, person_1, person_10, person_35, pe..."
1,(person_13),(person_1)
2,"(person_14, person_19)","(person_1, person_14, person_19)"
3,(person_19),(person_1)
4,(person_2),(person_1)
5,(person_26),(person_1)
6,(person_3),(person_1)
7,"(person_34, person_35)","(person_1, person_34, person_35)"
8,(person_35),(person_1)
9,"(person_1, person_5, person_4, person_9)","(person_1, person_5, person_9)"


In [29]:
def apply_rules(row):
    persons = set(row['person'])
    albums = set()

    # Sebagai loop utama untuk menerapkan rules pada data extract.
    # Untuk memeriksa apakah kombinasi antecedents dan consequents dalam grouped_rules terdapat dalam suatu foto.
    # iterasi setiap rule, lalu periksa apakah antecedents dan consequents ada dalam data persons suatu foto tersebut.
    # Mulai dari rules pertama, iterasi seluruh antecedents dan consequents, periksa apakah mereka ada dalam persons, lalu tambahkan ke albums jika cocok.
    for idx, (antecedents, consequents) in enumerate(grouped_rules):
        for antecedent in antecedents:
            for consequent in consequents:
                # Sebagai kondisi untuk memeriksa apakah antecedent dan consequent berbeda serta keduanya ada dalam data persons.
                # Untuk menentukan apakah aturan tersebut relevan dengan baris data, dan jika relevan, tambahkan indeks aturan tersebut ke dalam albums.
                # jika kondisi terpenuhi, tambahkan indeks rule ke albums dan break loop untuk menghindari duplikasi.
                if antecedent != consequent and consequent in persons and antecedent in persons:
                    albums.add(idx)
                    break  # Break to ensure only one album is added if the condition is met
    
    return list(albums)


In [30]:
# Apply the function to create albums
df_photos['albums'] = df_photos.apply(apply_rules, axis=1)

In [31]:
df_photos.albums

0       [4]
1       [4]
2       [4]
3        []
4       [4]
       ... 
636    [11]
637    [11]
638    [11]
639      []
640      []
Name: albums, Length: 641, dtype: object

In [32]:
df_photos

,photo_id,photo_name,person_id,person,albums
0,1,2013_08_30_19_35_IMG_3104.JPG,"2, 1","[person_2, person_1]",[4]
1,2,2013_08_30_19_37_IMG_2948.JPG,"1, 2","[person_1, person_2]",[4]
2,3,2013_08_30_19_37_IMG_2957.JPG,"1, 2","[person_1, person_2]",[4]
3,4,2014_01_01_09_20_IMG_2950.JPG,1,[person_1],[]
4,6,2014_01_01_10_18_IMG_2954.JPG,"2, 1","[person_2, person_1]",[4]
...,...,...,...,...,...
636,667,2021_01_02_17_22_IMG_4959.JPG,"1, 7, 21","[person_1, person_7, person_21]",[11]
637,668,2021_01_02_17_22_IMG_4960.JPG,"7, 1, 21","[person_7, person_1, person_21]",[11]
638,669,2021_01_02_17_22_IMG_4961.JPG,"21, 7, 1","[person_21, person_7, person_1]",[11]
639,670,2021_01_05_17_34_IMG_4995.JPG,26,[person_26],[]


### PENGELOMPOKKAN FOTO BERDASARKAN KLASIFIKASI

In [33]:
# Inisialisasi dictionary untuk menyimpan hasil
album_dict = {}

# Mengelompokkan photo_id berdasarkan album_id

# Sebagai loop utama untuk setiap baris di df_photos.
# Untuk mengelompokkan dan menyimpan informasi photo_id serta persons berdasarkan album_id ke dalam dictionary album_dict.
# loop setiap baris di df_photos, periksa album_id di dalam baris tersebut, lalu tambahkan informasi photo_id dan persons ke album_dict.
for _, row in df_photos.iterrows():
    album_ids = row['albums']
    for album_id in album_ids:
        if album_id not in album_dict:
            album_dict[album_id] = {'persons': set(), 'photo_ids': []}
        album_dict[album_id]['photo_ids'].append(row['photo_id'])
        album_dict[album_id]['persons'].update(row['person'])


In [34]:
def calculate_similarity(list1, list2):
    # Sebagai menghitung kesamaan antara dua list.
    # Digunakan untuk menentukan seberapa mirip 2 list berdasarkan elemen-elemen yang mereka miliki.
    # hitung kemunculan elemen dalam masing-masing list menggunakan Counter, lalu hitung elemen-elemen yang sama dan bagi dengan jumlah elemen terbesar dari kedua list.
    counter1 = Counter(list1)
    counter2 = Counter(list2)
    common_elements = sum((counter1 & counter2).values())
    total_elements = max(len(list1), len(list2))
    similarity = common_elements / total_elements
    return similarity

def cleanse_album_df(album_df):
    rows_to_drop = set()
    for (idx1, row1), (idx2, row2) in combinations(album_df.iterrows(), 2):
        album_similarity = calculate_similarity(row1['album'], row2['album'])
        photo_id_similarity = calculate_similarity(row1['photo_id'], row2['photo_id'])

        # Digunakan untuk mengidentifikasi album yang memiliki similarity lebih dari 30% sehingga dapat digabungkan.
        if album_similarity > 0.3:
            # print(f"Combining rows {idx1} and {idx2} based on similarities")
            if len(row1['photo_id']) > len(row2['photo_id']):
                larger_idx, smaller_idx = idx1, idx2
            else:
                larger_idx, smaller_idx = idx2, idx1

            # untuk mengelompokkan dan menggabungkan album yang lebih kecil ke dalam yang lebih besar.
            # setelah menentukan indeks, album yang lebih besar akan diperbarui dengan elemen dari album yang lebih kecil.
            larger_album_id = album_df.loc[larger_idx, 'photo_id']
            smaller_album_id = album_df.loc[smaller_idx, 'photo_id']

            # elemen-elemen unik dari album yang lebih kecil ditambahkan ke album yang lebih besar untuk menghindari duplikasi.
            larger_album = album_df.loc[larger_idx, 'album']
            smaller_album = album_df.loc[smaller_idx, 'album']
            
            # Menggabungkan photo_id unik dari album yang lebih kecil ke dalam album yang lebih besar
            unique_photo_ids = set(smaller_album_id) - set(larger_album_id)
            album_df.at[larger_idx, 'photo_id'] = list(set(larger_album_id) | unique_photo_ids)

            # memperbarui df dengan menggabungkan elemen-elemen unik dari album yang lebih kecil ke dalam yang lebih besar.
            # elemen-elemen dari album yang lebih kecil yang tidak ada di album yang lebih besar ditambahkan ke album yang lebih besar.
            unique_photo = set(smaller_album) - set(larger_album)
            album_df.at[larger_idx, 'album'] = list(set(larger_album) | unique_photo)
            
            rows_to_drop.add(smaller_idx)
    
    cleansed_df = album_df.drop(index=rows_to_drop).reset_index(drop=True)
    return cleansed_df


In [35]:
# Membuat DataFrame dari hasil
data_album = {
    'album_id': [],
    'album': [],
    'photo_id': []
}

In [36]:
for album_id, info in album_dict.items():
    # untuk memproses dan menyusun data album berdasarkan ID album dan person dan photo_ids.
    data_album['album_id'].append(album_id)
    data_album['album'].append(list(info['persons']))
    data_album['photo_id'].append(info['photo_ids'])


In [37]:
album_df = pd.DataFrame(data_album)
album_df

,album_id,album,photo_id
0,4,"[person_24, person_1, person_14, person_26, pe...","[1, 2, 3, 6, 9, 57, 58, 59, 60, 78, 79, 91, 92..."
1,9,"[person_20, person_1, person_62, person_47, pe...","[8, 20, 31, 37, 38, 50, 51, 53, 67, 68, 69, 83..."
2,10,"[person_20, person_5, person_9, person_1, pers...","[8, 31, 67, 68, 69, 83, 89, 94, 100, 102, 109,..."
3,13,"[person_20, person_1, person_62, person_47, pe...","[8, 20, 31, 37, 38, 50, 51, 53, 67, 68, 69, 83..."
4,24,"[person_20, person_5, person_9, person_1, pers...","[8, 31, 67, 68, 69, 83, 89, 94, 100, 102, 109,..."
5,25,"[person_20, person_1, person_62, person_47, pe...","[8, 20, 31, 37, 38, 50, 51, 53, 67, 68, 69, 83..."
6,26,"[person_20, person_5, person_9, person_1, pers...","[8, 31, 67, 68, 69, 83, 94, 100, 102, 109, 145..."
7,0,"[person_83, person_1, person_47, person_4, per...","[11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 2..."
8,7,"[person_34, person_83, person_9, person_1, per...","[11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 2..."
9,8,"[person_34, person_9, person_1, person_10, per...","[11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 2..."


In [38]:
cleansed_album_df = cleanse_album_df(album_df)
cleansed_album_df

,album_id,album,photo_id
0,4,"[person_24, person_1, person_14, person_26, pe...","[1, 2, 3, 6, 520, 9, 525, 526, 527, 541, 542, ..."
1,9,"[person_34, person_20, person_5, person_83, pe...","[8, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21..."
2,6,"[person_1, person_3, person_2]","[54, 308, 483, 484, 485, 486, 488, 489, 490, 4..."


### PEMBENTUKKAN ALBUM

In [39]:
# Sebagai membuat album berdasarkan album_id dan memindahkan foto-foto yang sesuai ke dalam folder album.
# untuk mengorganisir foto-foto ke dalam album yang terpisah berdasarkan ID album, mempermudah pengelolaan data foto.
def create_album(album_id, photos, df, pengguna_user):
 
    # Membuat folder album jika belum ada
    hasil_path = f'./hasil pengguna {pengguna_user}'
    album_path = f'{hasil_path}/{album_id}'
    os.makedirs(album_path, exist_ok=True)
    
    # Menyalin foto-foto ke dalam direktori album
    for photo_id in photos:
        # Digunakan untuk memastikan foto yang tepat dipindahkan ke album yang sesuai.
        photo_name = df.loc[df['photo_id'] == photo_id, 'photo_name'].values
        if len(photo_name) > 0:
            photo_name = photo_name[0]
            photo_path = f'./pengguna {pengguna_user}/{photo_name}'
            # Menyalin file foto ke direktori album
            shutil.copy(photo_path, f'{album_path}/{photo_name}')
    
    return album_path

def process_photos(df, pengguna_user):
    # Sebagai fungsi untuk memproses foto dan memindahkan foto-foto yang tidak memiliki album ke folder umum.

    # Membuat folder hasil_path jika belum ada
    hasil_path = f'./hasil pengguna {pengguna_user}'
    os.makedirs(hasil_path, exist_ok=True)

    # Menyalin foto-foto yang tidak memiliki album ke direktori hasil_path
    all_photo_ids = df['photo_id'].unique()
    album_photo_ids = [photo_id for ids in cleansed_album_df['photo_id'] for photo_id in ids]

    for photo_id in all_photo_ids:
        # dig unakan untuk memutuskan apakah foto perlu dipindahkan ke folder umum atau tetap di album.
        if photo_id not in album_photo_ids:
            photo_name = df.loc[df['photo_id'] == photo_id, 'photo_name'].values[0]
            photo_path = f'./pengguna {pengguna_user}/{photo_name}'
            # Menyalin file foto ke direktori hasil_path
            shutil.copy(photo_path, f'{hasil_path}/{photo_name}')


In [40]:
import time
# Menyimpan direktori album yang telah dibuat
albums = []

start_time = time.time()  # Mulai pencatatan waktu

# untuk memproses foto-foto yang tidak memiliki album dan menyalinnya ke folder umum.
process_photos(df_photos, pengguna_user)

# loop pada df kemudian membuat folder album dan menyalin foto, dan akhirnya menyimpan path album yang telah dibuat.
for _, row in cleansed_album_df.iterrows():
    album_id = row['album_id']
    photos = row['photo_id']
    album_path = create_album(album_id, photos, df_photos, pengguna_user)
    albums.append(album_path)

end_time = time.time()  # Akhiri pencatatan waktu

print("Album berhasil dibuat di:", albums)
print(f"Waktu komputasi: {end_time - start_time:.2f} detik")


Album berhasil dibuat di: ['./hasil pengguna 1/4', './hasil pengguna 1/9', './hasil pengguna 1/6']
Waktu komputasi: 3.20 detik


### MAKE SURE THE ALBUM

In [106]:
# Hitung jumlah baris yang memiliki nilai "220" dalam kolom 'People_id'
ppl = "230"
count = df_photos['People_id'].astype(str).str.contains(ppl).sum()

print(f'Jumlah baris yang memiliki nilai "{ppl}" dalam kolom People_id: {count}')

Jumlah baris yang memiliki nilai "230" dalam kolom People_id: 86
